In [1]:
import pandas as pd
from uuid import uuid4 as uuid
from enum import Enum
import requests

In [2]:
# # Lefty's calibration (2022-10-25)
# q1 = {#'freq_01': '3749584708.1745853',
#  'freq_01': '3747793170',
#  'init_duration': '0.0003',
#  'mw_amp180': '0.1573217116739378',
#  'ro_acq_delay': '3e-07',
#  'ro_acq_integration_time': '2.5e-06',
#  'ro_freq': '6741583902.617835',
#  'ro_pulse_amp': '0.00125',
#  'ro_pulse_delay': '4e-09',
#  'ro_pulse_duration': '3e-06'}
# q2 = {#'freq_01': '3305659156.82418',
#     'freq_01': '3304105194',
#  'init_duration': '0.0003',
#  'mw_amp180': '0.162628951043618',
#  'ro_acq_delay': '3e-07',
#  'ro_acq_integration_time': '2.5e-06',
#  'ro_freq': '6478209424.33459',
#  'ro_pulse_amp': '0.00500',
#  'ro_pulse_delay': '4e-09',
#  'ro_pulse_duration': '3e-06'}
# q3 = {#'freq_01': '3887617514.743307',
#     'freq_01': '3886156251',
#  'init_duration': '0.0003',
#  'mw_amp180': '0.14470129746405025',
#  'ro_acq_delay': '3e-07',
#  'ro_acq_integration_time': '2.5e-06',
#  'ro_freq': '7052143960.795639',
#  'ro_pulse_amp': '0.00500',
#  'ro_pulse_delay': '4e-09',
#  'ro_pulse_duration': '3e-06'}
# q4 = {#'freq_01': '3254451288.5693564',
#     'freq_01': '3252921704',
#  'init_duration': '0.0003',
#  'mw_amp180': '0.15721634788675287',
#  'ro_acq_delay': '3e-07',
#  'ro_acq_integration_time': '2.5e-06',
#  'ro_freq': '6578988854.977489',
#  'ro_pulse_amp': '0.00150',
#  'ro_pulse_delay': '4e-09',
#  'ro_pulse_duration': '3e-06'}
# q5 = {#'freq_01': '3983036942.3408036', # excitation freq
#     'freq_01': '3981583792',
#  'init_duration': '0.0003',
#  'mw_amp180': '0.15418024803095376', # convert to rabi freq
#  'ro_acq_delay': '3e-07', # tof
#  'ro_acq_integration_time': '2.5e-06', # integration time
#  'ro_freq': '6850488290.799288', # frequency of resonator
#  'ro_pulse_amp': '0.00150', # readout pulse amplitude
#  'ro_pulse_delay': '4e-09',
#  'ro_pulse_duration': '3e-06'} # readout pulse duration

In [3]:
# My calibration (2022-10-27)
q1 = {  #'freq_01': '3749584708.1745853',
    "freq_01": "3747793170",
    "init_duration": "0.0003",
    "mw_amp180": "0.045",
    "ro_acq_delay": "3e-07",
    "ro_acq_integration_time": "2.5e-06",
    "ro_freq": "6741583902.617835",
    "ro_pulse_amp": "0.00125",
    "ro_pulse_delay": "4e-09",
    "ro_pulse_duration": "3e-06",
}
q2 = {  #'freq_01': '3305659156.82418',
    "freq_01": "3304105194",
    "init_duration": "0.0003",
    "mw_amp180": "0.048",
    "ro_acq_delay": "3e-07",
    "ro_acq_integration_time": "2.5e-06",
    "ro_freq": "6478209424.33459",
    "ro_pulse_amp": "0.00500",
    "ro_pulse_delay": "4e-09",
    "ro_pulse_duration": "3e-06",
}
q3 = {  #'freq_01': '3887617514.743307',
    "freq_01": "3886156251",
    "init_duration": "0.0003",
    "mw_amp180": "0.042",
    "ro_acq_delay": "3e-07",
    "ro_acq_integration_time": "2.5e-06",
    "ro_freq": "7052143960.795639",
    "ro_pulse_amp": "0.00500",
    "ro_pulse_delay": "4e-09",
    "ro_pulse_duration": "3e-06",
}
q4 = {  #'freq_01': '3254451288.5693564',
    "freq_01": "3252921704",
    "init_duration": "0.0003",
    "mw_amp180": "0.046",
    "ro_acq_delay": "3e-07",
    "ro_acq_integration_time": "2.5e-06",
    "ro_freq": "6578988854.977489",
    "ro_pulse_amp": "0.00150",
    "ro_pulse_delay": "4e-09",
    "ro_pulse_duration": "3e-06",
}
q5 = {  #'freq_01': '3983036942.3408036', # excitation freq
    "freq_01": "3981583792",
    "init_duration": "0.0003",
    "mw_amp180": "0.047",  # convert to rabi freq
    "ro_acq_delay": "3e-07",  # tof
    "ro_acq_integration_time": "2.5e-06",  # integration time
    "ro_freq": "6850488290.799288",  # frequency of resonator
    "ro_pulse_amp": "0.00150",  # readout pulse amplitude
    "ro_pulse_delay": "4e-09",
    "ro_pulse_duration": "3e-06",
}  # readout pulse duration

In [4]:
qubits = (q1, q2, q3, q4, q5)

In [5]:
# for every qubit,
# create three documents:
# 1) resonator/spectroscopy with type resonator_calibrations
# 2) qubit/spectroscopy with type qubit_calibrations
# 3) qubit/rabi_oscillations with type qubit_calibrations

res_ids = list()
tt_ids = list()
rabi_ids = list()


class etype(Enum):
    resonator_spectroscopy = {
        "ro_freq": "frequency",
        "ro_pulse_amp": "square_amp",
        "ro_pulse_duration": "square_dur",
        "ro_acq_integration_time": "integration_time",
        "ro_acq_delay": "tof",
    }
    qubit_spectroscopy = {
        "freq_01": "frequency",
    }
    rabi_oscillations = {"mw_amp180": "frequency"}  # conversion: f = 1/(2AMP)


def new_document(e: etype, data: dict) -> dict:
    document = {
        "job_id": str(uuid()),
        "type": "resonator_calibrations"
        if "resonator" in e.name
        else "qubit_calibrations",
        "name": e.name,
    }

    key_mapping = e.value
    for k, m in key_mapping.items():
        if k == "mw_amp180":
            document[m] = 1 / (2 * float(data[k]))

        else:
            document[m] = float(data[k])

    if e.name == "rabi_oscillations":
        # document["gauss_dur"] = 52e-9
        # document["gauss_sig"] = 13e-9
        document["gauss_dur"] = 100e-9
        document["gauss_sig"] = 25e-9

    return document


docs = list()
for qb in qubits:
    docs.append(doc1 := new_document(etype.resonator_spectroscopy, qb))
    res_ids.append(doc1["job_id"])

    docs.append(doc2 := new_document(etype.qubit_spectroscopy, qb))
    tt_ids.append(doc2["job_id"])

    docs.append(doc3 := new_document(etype.rabi_oscillations, qb))
    rabi_ids.append(doc3["job_id"])

In [6]:
input("Post to MongoDB?")
requests.post("http://qtl-axean.mc2.chalmers.se:8002/calibrations", json=docs)
print("RESONATOR SPEC:")
print(res_ids)
print()
print("TWO TONE:")
print(tt_ids)
print()
print("RABI:")
print(rabi_ids)

Post to MongoDB?
RESONATOR SPEC:
['1fdedef3-b214-4e63-a61a-4d68c9a65fd1', '49d0d2c8-5da5-4ea6-b7d9-75625076bec4', '6bb2383a-89ce-45ad-9faf-fc6a596be15f', '63829e5d-691f-49a1-aac1-15349bf6ea1c', 'eb9d64ed-1a60-44b7-a083-5bf93e214c62']

TWO TONE:
['8464ba47-91f8-4f22-88ea-33a3492e3e56', '36ba2dd8-a703-4bc5-a6b7-625630a4082e', 'b6f0bfe5-3ebd-4b38-a9b7-0f03c36ead96', 'ecec2dc8-628c-4737-8682-2f2587416865', 'cd829aec-393d-4960-89aa-4ddbefa6ebe6']

RABI:
['fd66b448-0e51-4fbf-9d92-ce008056b495', 'cb9f781e-20db-455c-aa54-d003aa33ef59', '4b089e2f-49bd-44c8-8bb8-9a394f6ba584', '09e4d28b-618f-4b2d-b6e4-e9497220e043', 'bfc3aa31-e4c0-44e7-894c-c581554242d0']
